# Análise dos dados

A análise dos dados de sorteios de um consórcio de imóvel é essencial para entender o comportamento e as tendências dentro do grupo de consorciados. Essa análise permite identificar padrões de contemplação, prever os momentos mais propícios para o sorteio de determinados participantes e melhorar a gestão do consórcio como um todo. Além disso, auxilia na transparência e na tomada de decisões estratégicas, como ajustes nas regras do consórcio, proporcionando mais confiança e previsibilidade para os consorciados. Ao utilizar dados históricos e tendências, é possível aumentar a eficiência e a satisfação dos participantes.



## Os Dados

In [2]:
import pandas as pd
import plotly.express as px

In [3]:
id_cota = 2450765

In [4]:
df_contemplados = pd.read_parquet(f"./data/contemplados-{id_cota}.parquet")
df_opcoes_contemplacao = pd.read_parquet(f"./data/opcoes_contemplacao-{id_cota}.parquet")

### Agrupando dados relevantes

Aqui vamos fazer um dataframe onde a data da assembleia, cota, versao e quantidade de parcelas são únicos
entre contemplados e não contemplados. Precisamos juntar as informações e evitar possíveis duplicidades ou 
incoerências, onde por exemplo, um contemplado não faz parte das opções de contemplação.

Alem disso, vamos adicionar:

- **max_parcelas**: O número máximo possível de um lance que é o número de parcelas remanescentes do consórcio
- **max_viavel**: O número de parcelas de lance que seria equivalente a um financiamento imobilário.

In [58]:
df_contemplados["contemplado"] = 1
df_opcoes_contemplacao["contemplado"] = 0

colunas_relevantes = [
    "assembleia",
    "data_assembleia",
    "cota",
    "versao",
    "quantidadeParcelaLance",
    "modalidade",
    "contemplado",
]

df = pd.concat(
    [
        df_contemplados[colunas_relevantes],
        df_opcoes_contemplacao[colunas_relevantes],
    ]
)
df = df.groupby(colunas_relevantes[:-2]).max().reset_index()
df["max_parcelas"] = df["assembleia"].apply(lambda x: 216 - x)
df["max_viavel"] = df["assembleia"].apply(lambda x: 150 - x)
df["str_cota"] = df.apply(lambda x: f"{x['cota']}-{x['versao']}", axis=1)
# df

## Evolução das Cotas

Aqui utilizamos as opções de contamplação com apenas uma modalidade por cota.

### Opções de Contemplação

In [69]:
# uma seq. contemplacao por cota
df_chart = (
    df.groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index()
    .sort_values("str_cota", ascending=False)
)
fig = px.bar(
    df_chart,
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    labels={"data_assembleia": "Data da assembleia", "str_cota": "Numero de Cotas"},
    title=f"Opções de Contemplação por data de assembleia",
)
fig.show()

> **Entender**: São 650 cotas no grupo. Como o número de opções de contemplação em 1200 é táo maior?

### Cotas Ativas

In [70]:
# uma seq. contemplacao por cota
df_chart = (
    df.where(df["modalidade"] != "Sort. Canceladas")
    .groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index()
    .sort_values("str_cota", ascending=False)
)
fig = px.bar(
    df_chart,
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    labels={"data_assembleia": "Data da assembleia", "str_cota": "Numero de Cotas"},
    title=f"Opções de Contemplação (Sem Canceladas) por data de assembleia",
)
fig.show()

> **Entender**: Não existe uma linha decrescente nas opções de contemplação como seria esperado de um consórcio

## Contemplações

In [111]:
fig = px.bar(
    df[["data_assembleia", "modalidade", "str_cota"]]
    .where(df["contemplado"] == 1)
    .groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index(),
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    title=f"Contemplados por data de assembleia",
    labels={"str_cota": "Quantidade de cotas", "data_assembleia": "Data da assembleia"},
)
fig.show()

### Total de Contemplados

In [126]:
total_contemplados = {}

def cumsum_modalidade(x):
    data_assembleia = x["data_assembleia"]
    modalidade = x["modalidade"]
    contemplados = x["contemplado"]
    print(data_assembleia, modalidade, contemplados)
    por_modalidade = total_contemplados.get(modalidade)
    if por_modalidade is None:
        por_modalidade = {}
        por_modalidade[data_assembleia] = por_modalidade
    port_assembleia = por_modalidade.get(modalidade)
    if port_assembleia is None:
        port_assembleia = 0
    port_assembleia += contemplados
    por_modalidade[data_assembleia] = por_modalidade


df.apply(cumsum_modalidade, axis=1)
total_contemplados

2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sort. Canceladas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sort. Canceladas 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sort. Canceladas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Sort. Canceladas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sort. Canceladas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Sort. Canceladas 0
2020-10-15T00:00:00 Lance Fixo 0
2020-10-15T00:00:00 Sorteio Ativas 0
2020-10-15T00:00:00 Lance Fixo 0


{}

## Lances Livres

In [60]:
chart_df = (
    df[["data_assembleia", "max_parcelas", "max_viavel", "quantidadeParcelaLance"]]
    .where(df["modalidade"] == "Lance Livre")
    .groupby(["data_assembleia", "max_parcelas", "max_viavel"])
    .max()
    .reset_index()
)
# print(chart_df)
fig = px.bar(
    x=chart_df["data_assembleia"],
    y=chart_df["quantidadeParcelaLance"],
    labels={
        "data_assembleia": "Data da assembleia",
        "quantidadeParcelaLance": "Lance Livre",
    },
    title=f"Máximo Lance Livre Ofertado por Data de Assembléia",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_viavel"],
    mode="lines",
    name="Equivalente Financiamento de 13% a.a.",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_parcelas"],
    mode="lines",
    name="Máximo de Parcelas",
)

fig.show()

In [59]:
chart_df = (
    df[["data_assembleia", "max_parcelas", "max_viavel", "quantidadeParcelaLance"]]
    .where((df["modalidade"] == "Lance Livre") & (df["contemplado"] == 1))
    .groupby(["data_assembleia", "max_parcelas", "max_viavel"])
    .max()
    .reset_index()
)
# print(chart_df)
fig = px.bar(
    x=chart_df["data_assembleia"],
    y=chart_df["quantidadeParcelaLance"],
    labels={
        "data_assembleia": "Data da assembleia",
        "quantidadeParcelaLance": "Lance Livre",
    },
    title=f"Lance Livre Contemplado por Data de Assembléia",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_viavel"],
    mode="lines",
    name="Equivalente Financiamento de 13% a.a.",
)
fig.add_scatter(
    x=chart_df["data_assembleia"],
    y=chart_df["max_parcelas"],
    mode="lines",
    name="Máximo de Parcelas",
)

fig.show()

## Versão de Cotas

Como visto nos gráficos anteriores, é possível que uma cota tenha mais de uma versão para que as opções 
de conteplação somem mais do que as 650 cotas originais.

In [68]:
modalidades = df["modalidade"].unique()

for modalidade in modalidades:
    df_chart = (
        df.where(df["modalidade"] == modalidade)
        .groupby(["data_assembleia", "versao"])
        .count()
        .reset_index()
        .sort_values("versao")
    )
    fig = px.bar(
        df_chart,
        x="data_assembleia",
        y="str_cota",
        color="versao",
        labels={"data_assembleia": "Data da assembleia", "str_cota": "Numero de Cotas"},
        title=f"Opções de Contemplação {modalidade} por data de assembleia",
    )
    fig.show()

In [15]:
fig = px.bar(
    df.groupby(["data_assembleia", "versao"]).count().reset_index(),
    x="data_assembleia",
    y="cota",
    color="versao",
    labels={
        "data_assembleia": "Data da assembleia",
        "versao": "Versão",
        "cota": "Número de cotas",
    },
    title=f"Número de cotas por versão",
)
fig.show()

## Número de Contemplações  

In [16]:
import itertools

product = list(itertools.product(sorted(datas_assembleias), modalidades))
df_resumo = pd.DataFrame(product, columns=["data_assembleia", "modalidade"])
df_resumo.groupby(["data_assembleia", "modalidade"]).cumsum()

""
0
1
2
3
4
...
183
184
185
186


In [17]:
import numpy as np

df_count_contemplados = (
    df_contemplados[["data_assembleia", "modalidade", "str_cota"]]
    .where(df_contemplados["modalidade"] != "Sort. Canceladas")
    .sort_values("modalidade", ascending=False)
    .sort_values("data_assembleia")
    .groupby(["data_assembleia", "modalidade"])
    .count()
    .reset_index()
)
#df_count_contemplados["cumsum"] = df_count_contemplados["str_cota"].cumsum()
df_count_contemplados

,data_assembleia,modalidade,str_cota
0,2020-10-15T00:00:00,Lance Livre,1
1,2020-10-15T00:00:00,Sorteio Ativas,1
2,2020-11-12T00:00:00,Sorteio Ativas,1
3,2020-12-14T00:00:00,Lance Fixo,1
4,2020-12-14T00:00:00,Lance Livre,2
...,...,...,...
92,2024-07-12T00:00:00,Lance Livre,1
93,2024-07-12T00:00:00,Sorteio Ativas,1
94,2024-08-14T00:00:00,Lance Fixo,1
95,2024-08-14T00:00:00,Lance Livre,1


In [18]:
fig = px.bar(
    df_count_contemplados,
    x="data_assembleia",
    y="str_cota",
    color="modalidade",
    title=f"Total de contemplações de cotas ativas por data de assembleia",
    labels={"str_cota": "Quantidade de cotas", "data_assembleia": "Data da assembleia"},
)
fig.show()